In [29]:
import numpy as np
import pandas as pd

In [30]:
raw=pd.read_csv("data.csv")

In [31]:
df=raw[~raw["Review text"].isna()]

In [32]:
df.drop_duplicates(subset=["Review text"],inplace=True)

C:\Users\ravinder\AppData\Local\Temp\ipykernel_7732\375073976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=["Review text"],inplace=True)


In [33]:
df.reset_index(drop=True,inplace=True)

In [34]:
df.duplicated().sum()

0

In [35]:
df=df[["Review text"]]

In [36]:
df

,Review text
0,"Nice product, good quality, but price is now r..."
1,They didn't supplied Yonex Mavis 350. Outside ...
2,Worst product. Damaged shuttlecocks packed in ...
3,"Quite O. K. , but nowadays the quality of the..."
4,Over pricedJust â?¹620 ..from retailer.I didn'...
...,...
4978,Delivered before time but price is high from m...
4979,up to the mark but same is available in market...
4980,Nice delivery speedREAD MORE
4981,No complaints about the item . Its the best on...


In [37]:
df["Review text"]=df["Review text"].str.replace(".READ MORE","")

C:\Users\ravinder\AppData\Local\Temp\ipykernel_7732\2881371563.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Review text"]=df["Review text"].str.replace(".READ MORE","")


In [38]:
df["Review text"][0]

'Nice product, good quality, but price is now rising which is a bad sign. 800-850 was an affordable price, especially when we play everyday. So kindly help us out in terms of the price. Thank You'

In [39]:
import emoji
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [40]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [41]:
sw = stopwords.words('english')

In [42]:
rem=["no","not","aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]


In [43]:
for i in rem:
    sw.remove(i)

In [44]:
sw

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [45]:
def clean_text(doc):
    doc = doc.lower()
    doc=emoji.demojize(doc).replace(':','')
    doc = re.sub(r'[^A-z0-9 ]+','',doc)
    doc_list = doc.split()
    doc_list = [word for word in doc_list if word not in sw]
    doc_list = [lemmatizer.lemmatize(i) for i in doc_list]
    return " ".join(doc_list)

In [46]:
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [47]:
df['Text_cleaned'] = df['Review text'].progress_apply(clean_text)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4983/4983 [00:01<00:00, 4439.49it/s]


In [48]:
from nltk.sentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [49]:
def polarity(n):
    return analyzer.polarity_scores(n)["compound"]

df["polarity"]=df["Text_cleaned"].apply(polarity)

In [50]:
df

,Review text,Text_cleaned,polarity
0,"Nice product, good quality, but price is now r...",nice product good quality price rising bad sig...,0.9110
1,They didn't supplied Yonex Mavis 350. Outside ...,didnt supplied yonex mavis 350 outside cover y...,-0.4767
2,Worst product. Damaged shuttlecocks packed in ...,worst product damaged shuttlecock packed new b...,-0.9146
3,"Quite O. K. , but nowadays the quality of the...",quite k nowadays quality cork like not 3 5 yea...,0.8979
4,Over pricedJust â?¹620 ..from retailer.I didn'...,pricedjust 620 retaileri didnt understand wat ...,-0.1877
...,...,...,...
4978,Delivered before time but price is high from m...,delivered time price high marke,0.0000
4979,up to the mark but same is available in market...,mark available market le price,0.0000
4980,Nice delivery spee,nice delivery spee,0.4215
4981,No complaints about the item . Its the best on...,no complaint item best one way priced easily a...,0.4939


In [51]:
def sentiment(x):
    if x>0.0:
        return "positive"
    else:
        return "negative"
df["sentiment"]=df["polarity"].apply(sentiment)

In [52]:
df["sentiment"].value_counts()

positive    3041
negative    1942
Name: sentiment, dtype: int64

In [53]:
df

,Review text,Text_cleaned,polarity,sentiment
0,"Nice product, good quality, but price is now r...",nice product good quality price rising bad sig...,0.9110,positive
1,They didn't supplied Yonex Mavis 350. Outside ...,didnt supplied yonex mavis 350 outside cover y...,-0.4767,negative
2,Worst product. Damaged shuttlecocks packed in ...,worst product damaged shuttlecock packed new b...,-0.9146,negative
3,"Quite O. K. , but nowadays the quality of the...",quite k nowadays quality cork like not 3 5 yea...,0.8979,positive
4,Over pricedJust â?¹620 ..from retailer.I didn'...,pricedjust 620 retaileri didnt understand wat ...,-0.1877,negative
...,...,...,...,...
4978,Delivered before time but price is high from m...,delivered time price high marke,0.0000,negative
4979,up to the mark but same is available in market...,mark available market le price,0.0000,negative
4980,Nice delivery spee,nice delivery spee,0.4215,positive
4981,No complaints about the item . Its the best on...,no complaint item best one way priced easily a...,0.4939,positive


In [54]:
dict(zip(df["Review text"],df["sentiment"]))

{'Nice product, good quality, but price is now rising which is a bad sign. 800-850 was an affordable price, especially when we play everyday. So kindly help us out in terms of the price. Thank You': 'positive',
 "They didn't supplied Yonex Mavis 350. Outside cover was Yonex Ad inside was a cheapest....  Sad to hear this": 'negative',
 "Worst product. Damaged shuttlecocks packed in new box. It's not a original yonex product. Don't buy.flipkart platform is chosen to fraud the buyers": 'negative',
 'Quite O. K. , but nowadays  the quality of the corks like not as before 3 to 5 years back.. I am using MAVIS 350 for more than 15 years quality of corks was very very good at that times, but now I am not getting the quality corks as like before, rate of corks also too much now, I am  very sorry to say like this, but in my experience , my Statment is very true to   my knowledg': 'positive',
 "Over pricedJust â?¹620 ..from retailer.I didn't understand.. Wat is d advantage of buying dis frm flipk

In [55]:
df

,Review text,Text_cleaned,polarity,sentiment
0,"Nice product, good quality, but price is now r...",nice product good quality price rising bad sig...,0.9110,positive
1,They didn't supplied Yonex Mavis 350. Outside ...,didnt supplied yonex mavis 350 outside cover y...,-0.4767,negative
2,Worst product. Damaged shuttlecocks packed in ...,worst product damaged shuttlecock packed new b...,-0.9146,negative
3,"Quite O. K. , but nowadays the quality of the...",quite k nowadays quality cork like not 3 5 yea...,0.8979,positive
4,Over pricedJust â?¹620 ..from retailer.I didn'...,pricedjust 620 retaileri didnt understand wat ...,-0.1877,negative
...,...,...,...,...
4978,Delivered before time but price is high from m...,delivered time price high marke,0.0000,negative
4979,up to the mark but same is available in market...,mark available market le price,0.0000,negative
4980,Nice delivery spee,nice delivery spee,0.4215,positive
4981,No complaints about the item . Its the best on...,no complaint item best one way priced easily a...,0.4939,positive


In [56]:
#df.drop("polarity",axis=1,inplace=True)

In [58]:
df.to_csv("reviews_sentiment",index=False)

In [1]:
import joblib

In [2]:
bayes = joblib.load('naive_bayes.pkl')
log = joblib.load('naive_bayes.pkl')
dec = joblib.load('naive_bayes.pkl')

In [3]:
rev="😍👌 Absolutely fantastic! The food was delectable, and the service was top-notch. The staff went above and beyond to ensure we had a memorable dining experience. Can't wait to go back!"

In [5]:
prediction = log.predict([rev])

print("Prediction:", prediction)

Prediction: ['positive']
